# Mock SQL

In [35]:
import numpy as np
from collections import defaultdict

In [59]:
class Table:
    
    def __init__(self, columns):
        self.columns = columns # i.e. a list of column names.
        self.rows = []
    
    def __repr__(self):
        # pretty print.
        return str(self.columns) + "\n" + "\n".join(map(str, self.rows))
    
    def insert(self, rowValues):
        if len(rowValues) != len(self.columns):
            raise TypeError("wrong number of elements")
        rowDict = dict(zip(self.columns, rowValues))
        self.rows.append(rowDict)
    
    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, newValue in updates.iteritems():
                    row[column] = newValue
    
    def delete(self, predicate=lambda row:True):
        self.rows = [row for row in self.rows if not(predicate(row))]
        
    def select(self, keepColumns=None, additionalColumns=None):
        # keepColumns: select by column names; select all if = None.
        # additionalColumns: containts new columns and how the values in the new columns should be computed.
        if keepColumns is None:
            keepColumns = self.columns
        if additionalColumns is None:
            additionalColumns = {}
        resultTable = Table(keepColumns + additionalColumns.keys())
        for row in self.rows:
            newRow = [row[column] for column in keepColumns]
            for columnName,calculation in additionalColumns.iteritems():
                newRow.append(calculation(row))
            resultTable.insert(newRow)
        return resultTable
    
    def where(self, predicate=lambda row:True):
        # return rows that satisfy the supplied predicate.
        whereTable = Table(self.columns)
        whereTable.rows = filter(predicate, self.rows)
        return whereTable
    
    def limit(self, numRows):
        # return only the first numRow rows.
        limitTable = Table(self.columns)
        limitTable.rows = self.rows[:numRows]
        return limitTable
    
    def group_by(self, groupByColumns, aggregates, having=None):
        groupedRows = defaultdict(list)
        for row in self.rows:
            key = tuple(row[column] for column in groupByColumns)
            groupedRows[key].append(row)
        resultTable = Table(groupByColumns + aggregates.keys())
        for key,rows in groupedRows.iteritems():
            if having is None or having(rows):
                newRow = list(key)
                for aggregateName,aggregate_fn in aggregates.iteritems():
                    newRow.append(aggregate_fn(rows))
                resultTable.insert(newRow)
        return resultTable
    
    def order_by(self, order):
        newTable = self.select()
        newTable.rows.sort(key=order)
        return newTable
        

In [60]:
# ADD ENTRIES
users = Table(["user_id", "name", "num_friends"])
users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])
print users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


In [39]:
# UPDATE DATABASE
users.update({'num_friends': 3}, lambda row: row['user_id']==1) # user1, i.e. Dunn, acquires 1 new friend.
print users.rows[1]

{'user_id': 1, 'name': 'Dunn', 'num_friends': 3}


In [40]:
# DELETE (ENTRIES FROM / DATABASE)
users.delete(lambda row: row["user_id"]==1) # delete user1, i.e. Dunn.
# users.delete() # delete everything, dangerous.
print users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}


In [41]:
# GET A NEW TABLE WITH MODIFICATION ON OLD
users.select(keepColumns=["name", "num_friends"],
             additionalColumns = {"facebook_friends": lambda row: row["num_friends"] + np.random.randint(10)})

['name', 'num_friends', 'facebook_friends']
{'facebook_friends': 0, 'name': 'Hero', 'num_friends': 0}
{'facebook_friends': 12, 'name': 'Sue', 'num_friends': 3}
{'facebook_friends': 9, 'name': 'Chi', 'num_friends': 3}
{'facebook_friends': 7, 'name': 'Thor', 'num_friends': 3}
{'facebook_friends': 7, 'name': 'Clive', 'num_friends': 2}
{'facebook_friends': 8, 'name': 'Hicks', 'num_friends': 3}
{'facebook_friends': 9, 'name': 'Devin', 'num_friends': 2}
{'facebook_friends': 8, 'name': 'Kate', 'num_friends': 2}
{'facebook_friends': 9, 'name': 'Klein', 'num_friends': 3}
{'facebook_friends': 4, 'name': 'Jen', 'num_friends': 1}

In [42]:
users.where(lambda row: row["num_friends"]>2) 

['user_id', 'name', 'num_friends']
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}

In [47]:
users.where(lambda row: row["name"]=="Hero").select(keepColumns=["user_id"]) 
    # get the id of a user with the name "Dunn".

['user_id']
{'user_id': 0}

In [48]:
users.limit(3)

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}

In [58]:
# GROUP BY
users.group_by(groupByColumns=["name"], 
               aggregates={"first_letter_of_name": lambda rows:[row["name"][0] if row["name"] else "" for row in rows]})


['name', 'first_letter_of_name']
{'name': 'Thor', 'first_letter_of_name': ['T']}
{'name': 'Devin', 'first_letter_of_name': ['D']}
{'name': 'Jen', 'first_letter_of_name': ['J']}
{'name': 'Sue', 'first_letter_of_name': ['S']}
{'name': 'Kate', 'first_letter_of_name': ['K']}
{'name': 'Chi', 'first_letter_of_name': ['C']}
{'name': 'Hero', 'first_letter_of_name': ['H']}
{'name': 'Klein', 'first_letter_of_name': ['K']}
{'name': 'Clive', 'first_letter_of_name': ['C']}
{'name': 'Hicks', 'first_letter_of_name': ['H']}

In [62]:
# ORDER BY
users.order_by(lambda row: -row["num_friends"]) # for ascending, drop "-".

['user_id', 'name', 'num_friends']
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}

In [63]:
# SQL bores the heck out of me...